# Reinforcement learning 

In this Python notebook, we will have you implement a simple reinforcement learning agent for the AI gym mountain car problem. Please first have a look at the description of the task here: <A HREF="https://github.com/openai/gym/wiki/MountainCar-v0" TARGET="_blank">Description</A>

## Heuristic agent
Before we dive into the use of reinforcement learning of the mountain car task, you will first make a strategy by hand yourself. There are two very good reasons for doing so. 

First, making such a "heuristic" agent will give you a good idea of how difficult the task is, given the observations and actions of the agent. In the case of the mountain car task, the agent has to reach a goal position (located at position $0.5$), by choosing from discrete actions $\{0,1,2\}$ corresponding to $\{$ push left, no acceleration, push right $\}$. The starting position is in the middle, at position $-0.5$. The left end of the environment has position $-1.2$. The velocity of the agent is limited to the interval $[-0.07, 0.07]$.

Second, making a heuristic agent gives you a good idea of the difficulty of the task, and what kind of solution you might expect the machine learning method to find (reinforcement learning in the current notebook).

We will experiment with the original formulation of the car mountain car task. The class we will use is `CMC_original`, which is the same as the normal AI gym version, but with an adapted render-function in order to be able to show the graphics in a Binder notebook. The full code, imported with `import run_cart` in the code block below. You can find it <A HREF="https://github.com/guidoAI/RL_cart_notebook/blob/master/run_cart.py" TARGET="_blank">here</A>. 

<FONT COLOR="red">Exercise 1.</FONT>
<OL>
    <LI>First run the following block, in which an agent is run that takes random actions. Does it succeed in performing the task? Why so? </LI>
    <LI>If we want the agent to move to the right, why then not just drive there? Try this strategy out by replacing the random action in the `act` function with 2, which means to just accelerate to the right. What do you observe, and why?</LI>
    <LI>Write a heuristic method to solve the task. Only make use of the position and velocity in the current time step, as this is what we are going to give to the reinforcement learning agent. Are you able to solve the task? Is your solution optimal you think? Do you think that this will be easy to learn with reinforcement learning?</LI>
</OL>

In [2]:
%matplotlib inline

import os
import matplotlib
from matplotlib import pyplot as plt
import run_cart
import gym
import numpy as np
import random

# definition of the agent class
class heuristic_agent(object):

    def act(self, observation, reward, done):
        """ - observation[0] = position
            - observation[1] = velocity
            - reward = the reward received from the environment
            - done = whether the agent reached the goal
        """
        position = observation[0]
        velocity = observation[1]
        
        # REPLACE THIS CODE WITH A FIXED ACTION OR YOUR HEURISTIC:
        action = random.randint(0,2)
        
        return action

# create the agent:
agent = heuristic_agent()
# apply the agent to the task
reward, rewards = run_cart.run_cart_discrete(agent, env=run_cart.CMC_original(), graphics=True)
print('Reward = ' + str(reward))

Reward = -200.0


## Q-learning
For this notebook, you will implement a very elementary reinforcement learning method: Q-learning. Please first read up on this method <A HREF="https://en.wikipedia.org/wiki/Q-learning" TARGET="_blank">here</A>. In particular, we will investigate the use of _tabular_ Q-learning, in which the algorithm finds the values for all state-action pairs in the matrix. Since our state is two-dimensional (position and velocity), the state-action space can be discretized without leading to an overly large table. In the code block below, we create the table in the `__init__` function of `Q_learning_agent`, using 10 grid cells per state variable. Also have a look at the start of the `act` function. When we get the two-dimensional state (observation), we transform it to a single state number (row index in the Q state-action table). The table has three columns for the three actions.

<FONT COLOR="red">Exercise 2.</FONT>
<OL>
    <LI>Fill in the Q-update function in the `act` function. Remember that variables of the agent object have to be reffered to with `self`  like `self.alpha` and `self.Q`.</LI>
    <LI>Write the code to take actions under an $\epsilon$-greedy policy, where $\epsilon$ is `p_explore` in the code.
    </LI>
    <LI>Run the code to have the agent learn. Is it able to do the task (to check this, run the code block two blocks below in order to run the agent without exploratory actions)? Why / why not? If it does not work, can you then change the code at the bottom to make the agent learn the task properly?</LI>
</OL>

In [ ]:
class Q_learning_agent(object):
    """Simple Q-learning agent for the MountainCarv0 task
       https://en.wikipedia.org/wiki/Q-learning
    """

    n_actions = 3

    def __init__(self, min_speed, max_speed, min_position, max_position, alpha = 0.1, gamma = 0.9, p_explore = 0.1):
        
        # number of grids per state variable
        self.n_grid = 10
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.speed_step = (max_speed - min_speed) / self.n_grid
        self.min_position = min_position
        self.max_position = max_position
        self.position_step = (max_position - min_position) / self.n_grid
        # discretizing the 2-variable state results in this number of states:
        self.n_states = int(self.n_grid**2)
        # make an empty Q-matrix
        self.Q = np.zeros([self.n_states, self.n_actions])
        # initialize previous state and action
        self.previous_state = 0
        self.previous_action = 0
        # learning rate
        self.alpha = alpha
        # discount factor:
        self.gamma = gamma
        # e-greedy, p_explore results in a random action:
        self.p_explore = p_explore

    def act(self, observation, reward, done, verbose = False):
        
        # Determine the new state:
        pos = observation[0]
        if(pos > self.max_position):
            pos = self.max_position
        elif(pos < self.min_position):
            pos = self.min_position
        obs_pos = int((pos - self.min_position) // self.position_step)                
        vel = observation[1]
        if(vel > self.max_speed):
            vel = self.max_speed
        elif(vel < self.min_speed):
            vel = self.min_speed
        obs_vel = int((vel - self.min_speed) // self.speed_step)
        new_state = obs_pos * self.n_grid + obs_vel
        
        if(verbose):
            print(f'Velocity {observation[1]}, position {observation[0]}, (grid {self.speed_step}, \
                          {self.position_step}), state = {new_state}')
        
        # ************************************************
        # FILL IN YOUR CODE HERE FOR THE Q-UPDATE FUNCTION
        # ************************************************
        
        # Update the Q-matrix:
        # self.Q[..., ...] =  ...
        
        # determine the new action:
        # FILL IN YOUR CODE FOR SELECTING AN ACTION
        
        # update previous state and action
        self.previous_state = new_state
        self.previous_action = action        
        
        # return the action
        return action

    
env=run_cart.CMC_original()

# set up off-policy learning with p_explore = 1
max_velocity = env.max_speed
min_velocity = -max_velocity
agent = Q_learning_agent(min_velocity, max_velocity, env.min_position, env.max_position, \
                         alpha = 0.20, gamma = 0.95, p_explore = 1.0)
n_episodes = 1000
reward, rewards = run_cart.run_cart_discrete(agent, env=env, graphics=False, n_episodes=n_episodes)
print('Reward per episode = ' + str(reward / n_episodes))

In [ ]:
n_episodes = 1
agent.p_explore = 0
agent.alpha = 0
reward, rewards = run_cart.run_cart_discrete(agent, env=env, graphics=True, n_episodes=n_episodes)
print(f'Reward trained agent {reward}')

## Answers

<FONT COLOR="red">Exercise 1.</FONT>
<OL>
    <LI>It typically does not succeed. For example, just when it speeds up to the right it then randomly accelerates to the left, and will not reach the goal.</LI>
    <LI>The agent just never gets high enough on the up-slope. The exerted force is not sufficient by itself to counter gravity. The agent will likely have to swing up and down to gain more and more speed, which will finally bring it over the right hill.</LI>
    <LI>The following code block has a solution to the task consisting of a few if / else statements. It always succeeds when starting at the start location. Whether it is the optimal solution is not clear, but it is clear that the state space can be separated quite easily and linked to the appropriate actions. It seems that the task is relatively easy to perform and learning algorithms should be able to do it.</LI>
</OL>

In [ ]:
class heuristic_agent(object):
    """Guido's heuristic agent"""

    def act(self, observation, reward, done):
        position = observation[0]
        velocity = observation[1]
        if position < -0.5:
            if velocity < -0.01:
                action = 0
            else:
                action = 2
        else:
            if velocity > 0.01:
                action = 2
            else:
                action = 0
        return action
    
agent = heuristic_agent()
reward, rewards = run_cart.run_cart_discrete(agent, env=run_cart.CMC_original(), graphics=True)
print('Reward = ' + str(reward))

<FONT COLOR="red">Exercise 2.</FONT>
<OL>
    <LI>The code for the update equation is as follows: 
        `self.Q[self.previous_state, self.previous_action] +=  self.alpha *  (reward + self.gamma * max(self.Q[new_state, :]) - self.Q[self.previous_state, self.previous_action])`</LI>
    <LI>A possible implementation is:
        `if(random.random() < self.p_explore): action = random.randint(0, self.n_actions-1) else: action = np.argmax(self.Q[new_state, :])`</LI>
    <LI>In order to solve it, I made a learning scheme in three parts: off-policy learning with random actions, on-policy learning with less exploration, and on-policy learning with a lower learning rate and even less exploration.</LI>
</OL>

In [ ]:
class Q_learning_agent(object):
    """Simple Q-learning agent for the MountainCarv0 task
       https://en.wikipedia.org/wiki/Q-learning
    """

    n_actions = 3

    def __init__(self, min_speed, max_speed, min_position, max_position, alpha = 0.1, gamma = 0.9, p_explore = 0.1):
        
        # number of grids per state variable
        self.n_grid = 10
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.speed_step = (max_speed - min_speed) / self.n_grid
        self.min_position = min_position
        self.max_position = max_position
        self.position_step = (max_position - min_position) / self.n_grid
        # discretizing the 2-variable state results in this number of states:
        self.n_states = int(self.n_grid**2)
        # make an empty Q-matrix
        self.Q = np.zeros([self.n_states, self.n_actions])
        # initialize previous state and action
        self.previous_state = 0
        self.previous_action = 0
        # learning rate
        self.alpha = alpha
        # discount factor:
        self.gamma = gamma
        # e-greedy, p_explore results in a random action:
        self.p_explore = p_explore

    def act(self, observation, reward, done, verbose = False):
        
        # Determine the new state:
        pos = observation[0]
        if(pos > self.max_position):
            pos = self.max_position
        elif(pos < self.min_position):
            pos = self.min_position
        obs_pos = int((pos - self.min_position) // self.position_step)                
        vel = observation[1]
        if(vel > self.max_speed):
            vel = self.max_speed
        elif(vel < self.min_speed):
            vel = self.min_speed
        obs_vel = int((vel - self.min_speed) // self.speed_step)
        new_state = obs_pos * self.n_grid + obs_vel
        
        if(verbose):
            print(f'Velocity {observation[1]}, position {observation[0]}, (grid {self.speed_step}, \
                          {self.position_step}), state = {new_state}')
        
        # Update the Q-matrix:
        self.Q[self.previous_state, self.previous_action] +=  self.alpha * \
            (reward + self.gamma * max(self.Q[new_state, :]) - self.Q[self.previous_state, self.previous_action])
        
        # determine the new action:
        if(random.random() < self.p_explore):
            action = random.randint(0, self.n_actions-1)
        else:
            action = np.argmax(self.Q[new_state, :])
        
        # update previous state and action
        self.previous_state = new_state
        self.previous_action = action        
        
        # return the action
        return action

    
env=run_cart.CMC_original()

# set up off-policy learning with p_explore = 1
max_velocity = env.max_speed
min_velocity = -max_velocity
agent = Q_learning_agent(min_velocity, max_velocity, env.min_position, env.max_position, \
                         alpha = 0.20, gamma = 0.95, p_explore = 1.0)
n_episodes = 1000
reward, rewards = run_cart.run_cart_discrete(agent, env=env, graphics=False, n_episodes=n_episodes)
print('Reward per episode = ' + str(reward / n_episodes))

# on-policy now with e-greedy
agent.p_explore = 0.05
reward, rewards = run_cart.run_cart_discrete(agent, env=env, graphics=False, n_episodes=n_episodes)
print('Reward per episode = ' + str(reward / n_episodes))

n_episodes = 100
agent.alpha = 0.05
agent.p_explore = 0.02
reward, rewards = run_cart.run_cart_discrete(agent, env=env, graphics=False, n_episodes=n_episodes)
print('Reward per episode = ' + str(reward / n_episodes))
